In [1]:
import re
import random

data_path = "model/human_text.txt"
data_path2 = "model/robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hola',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line.lower())) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line.lower())) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [2]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [3]:
print(pairs[:5])
print(input_docs[:5])

[('hola buenos días', 'buenos días en que puedo servirle'), ('tienes domicilio', 'claro que si que deseas pedir a domicilio'), ('que cuesta la arepa de huevo', 'la arepa tiene un costo de 2000 pesos sin incluir el domicilio'), ('y vendes chocolatico caliente', 'si el chocolate tiene un costo de 1000 sin leche'), ('y con leche', 'con leche tiene un costo de 2000 pesos')]
['hola buenos días', 'tienes domicilio', 'que cuesta la arepa de huevo', 'y vendes chocolatico caliente', 'y con leche']


In [4]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model, save_model

#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 800
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

training_model.save('model/training_model.h5')
training_model.save_weights('model/training_weights.hdf5')

Epoch 1/800
21/21 [==============================] - 9s 133ms/step - loss: 2.0291 - accuracy: 0.0381 - val_loss: 1.8722 - val_accuracy: 0.0456
Epoch 2/800
21/21 [==============================] - 1s 47ms/step - loss: 1.7375 - accuracy: 0.0489 - val_loss: 1.8769 - val_accuracy: 0.0456
Epoch 3/800
21/21 [==============================] - 1s 48ms/step - loss: 1.7658 - accuracy: 0.0530 - val_loss: 1.8830 - val_accuracy: 0.0480
Epoch 4/800
21/21 [==============================] - 1s 49ms/step - loss: 1.6219 - accuracy: 0.0526 - val_loss: 1.8976 - val_accuracy: 0.0542
Epoch 5/800
21/21 [==============================] - 1s 49ms/step - loss: 1.6874 - accuracy: 0.0614 - val_loss: 1.9111 - val_accuracy: 0.0558
Epoch 6/800
21/21 [==============================] - 1s 47ms/step - loss: 1.6237 - accuracy: 0.0599 - val_loss: 1.9108 - val_accuracy: 0.0566
Epoch 7/800
21/21 [==============================] - 1s 48ms/step - loss: 1.6701 - accuracy: 0.0606 - val_loss: 1.9317 - val_accuracy: 0.0676
Epoch

21/21 [==============================] - 1s 47ms/step - loss: 0.8435 - accuracy: 0.2040 - val_loss: 1.7694 - val_accuracy: 0.1407
Epoch 59/800
21/21 [==============================] - 1s 48ms/step - loss: 0.9202 - accuracy: 0.1975 - val_loss: 1.7847 - val_accuracy: 0.1407
Epoch 60/800
21/21 [==============================] - 1s 47ms/step - loss: 0.8654 - accuracy: 0.2037 - val_loss: 1.7891 - val_accuracy: 0.1360
Epoch 61/800
21/21 [==============================] - 1s 47ms/step - loss: 0.8881 - accuracy: 0.2038 - val_loss: 1.7949 - val_accuracy: 0.1415
Epoch 62/800
21/21 [==============================] - 1s 48ms/step - loss: 0.8830 - accuracy: 0.2115 - val_loss: 1.7982 - val_accuracy: 0.1297
Epoch 63/800
21/21 [==============================] - 1s 47ms/step - loss: 0.8884 - accuracy: 0.2205 - val_loss: 1.7867 - val_accuracy: 0.1415
Epoch 64/800
21/21 [==============================] - 1s 47ms/step - loss: 0.8949 - accuracy: 0.2043 - val_loss: 1.7889 - val_accuracy: 0.1423
Epoch 65/800

21/21 [==============================] - 1s 47ms/step - loss: 0.5038 - accuracy: 0.2703 - val_loss: 1.7749 - val_accuracy: 0.1533
Epoch 116/800
21/21 [==============================] - 1s 47ms/step - loss: 0.4505 - accuracy: 0.2925 - val_loss: 1.7467 - val_accuracy: 0.1517
Epoch 117/800
21/21 [==============================] - 1s 47ms/step - loss: 0.4858 - accuracy: 0.2855 - val_loss: 1.7718 - val_accuracy: 0.1557
Epoch 118/800
21/21 [==============================] - 1s 47ms/step - loss: 0.4650 - accuracy: 0.2894 - val_loss: 1.8117 - val_accuracy: 0.1509
Epoch 119/800
21/21 [==============================] - 1s 47ms/step - loss: 0.4707 - accuracy: 0.2908 - val_loss: 1.7866 - val_accuracy: 0.1525
Epoch 120/800
21/21 [==============================] - 1s 48ms/step - loss: 0.4779 - accuracy: 0.2949 - val_loss: 1.7976 - val_accuracy: 0.1494
Epoch 121/800
21/21 [==============================] - 1s 47ms/step - loss: 0.4879 - accuracy: 0.2868 - val_loss: 1.7804 - val_accuracy: 0.1549
Epoch 

Epoch 172/800
21/21 [==============================] - 1s 47ms/step - loss: 0.3473 - accuracy: 0.3081 - val_loss: 1.8397 - val_accuracy: 0.1541
Epoch 173/800
21/21 [==============================] - 1s 48ms/step - loss: 0.3366 - accuracy: 0.3148 - val_loss: 1.8312 - val_accuracy: 0.1557
Epoch 174/800
21/21 [==============================] - 1s 47ms/step - loss: 0.3089 - accuracy: 0.3189 - val_loss: 1.8254 - val_accuracy: 0.1580
Epoch 175/800
21/21 [==============================] - 1s 48ms/step - loss: 0.3344 - accuracy: 0.3094 - val_loss: 1.8300 - val_accuracy: 0.1572
Epoch 176/800
21/21 [==============================] - 1s 48ms/step - loss: 0.3285 - accuracy: 0.3310 - val_loss: 1.8304 - val_accuracy: 0.1564
Epoch 177/800
21/21 [==============================] - 1s 48ms/step - loss: 0.3248 - accuracy: 0.3151 - val_loss: 1.8156 - val_accuracy: 0.1604
Epoch 178/800
21/21 [==============================] - 1s 47ms/step - loss: 0.3430 - accuracy: 0.3191 - val_loss: 1.7875 - val_accuracy:

Epoch 229/800
21/21 [==============================] - 1s 47ms/step - loss: 0.2816 - accuracy: 0.3484 - val_loss: 1.9018 - val_accuracy: 0.1541
Epoch 230/800
21/21 [==============================] - 1s 48ms/step - loss: 0.2915 - accuracy: 0.3398 - val_loss: 1.9035 - val_accuracy: 0.1509
Epoch 231/800
21/21 [==============================] - 1s 47ms/step - loss: 0.2645 - accuracy: 0.3252 - val_loss: 1.9550 - val_accuracy: 0.1478
Epoch 232/800
21/21 [==============================] - 1s 47ms/step - loss: 0.2922 - accuracy: 0.3314 - val_loss: 1.9308 - val_accuracy: 0.1517
Epoch 233/800
21/21 [==============================] - 1s 47ms/step - loss: 0.2696 - accuracy: 0.3421 - val_loss: 1.9200 - val_accuracy: 0.1502
Epoch 234/800
21/21 [==============================] - 1s 47ms/step - loss: 0.2961 - accuracy: 0.3401 - val_loss: 1.9354 - val_accuracy: 0.1525
Epoch 235/800
21/21 [==============================] - 1s 48ms/step - loss: 0.2761 - accuracy: 0.3323 - val_loss: 1.9251 - val_accuracy:

...

Epoch 571/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1219 - accuracy: 0.3509 - val_loss: 2.0992 - val_accuracy: 0.1384
Epoch 572/800
21/21 [==============================] - 1s 48ms/step - loss: 0.1250 - accuracy: 0.3723 - val_loss: 2.1063 - val_accuracy: 0.1407
Epoch 573/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1065 - accuracy: 0.3509 - val_loss: 2.0555 - val_accuracy: 0.1431
Epoch 574/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1185 - accuracy: 0.3491 - val_loss: 2.0323 - val_accuracy: 0.1502
Epoch 575/800
21/21 [==============================] - 1s 46ms/step - loss: 0.1177 - accuracy: 0.3585 - val_loss: 2.0619 - val_accuracy: 0.1439
Epoch 576/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1164 - accuracy: 0.3602 - val_loss: 2.0743 - val_accuracy: 0.1360
Epoch 577/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1109 - accuracy: 0.3615 - val_loss: 2.0926 - val_accuracy:

Epoch 628/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1042 - accuracy: 0.3604 - val_loss: 2.0801 - val_accuracy: 0.1439
Epoch 629/800
21/21 [==============================] - 1s 48ms/step - loss: 0.1038 - accuracy: 0.3642 - val_loss: 2.0880 - val_accuracy: 0.1439
Epoch 630/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1035 - accuracy: 0.3532 - val_loss: 2.1065 - val_accuracy: 0.1447
Epoch 631/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1039 - accuracy: 0.3546 - val_loss: 2.0887 - val_accuracy: 0.1399
Epoch 632/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0919 - accuracy: 0.3633 - val_loss: 2.1046 - val_accuracy: 0.1423
Epoch 633/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1192 - accuracy: 0.3685 - val_loss: 2.1177 - val_accuracy: 0.1447
Epoch 634/800
21/21 [==============================] - 1s 47ms/step - loss: 0.1032 - accuracy: 0.3442 - val_loss: 2.1169 - val_accuracy:

Epoch 685/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0871 - accuracy: 0.3578 - val_loss: 2.1324 - val_accuracy: 0.1360
Epoch 686/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0894 - accuracy: 0.3521 - val_loss: 2.1439 - val_accuracy: 0.1344
Epoch 687/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0840 - accuracy: 0.3540 - val_loss: 2.1630 - val_accuracy: 0.1344
Epoch 688/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0915 - accuracy: 0.3603 - val_loss: 2.1213 - val_accuracy: 0.1384
Epoch 689/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0900 - accuracy: 0.3512 - val_loss: 2.1550 - val_accuracy: 0.1313
Epoch 690/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0871 - accuracy: 0.3554 - val_loss: 2.1555 - val_accuracy: 0.1321
Epoch 691/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0897 - accuracy: 0.3606 - val_loss: 2.1566 - val_accuracy:

Epoch 742/800
21/21 [==============================] - 1s 48ms/step - loss: 0.0744 - accuracy: 0.3576 - val_loss: 2.1341 - val_accuracy: 0.1360
Epoch 743/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0794 - accuracy: 0.3577 - val_loss: 2.1569 - val_accuracy: 0.1266
Epoch 744/800
21/21 [==============================] - 1s 48ms/step - loss: 0.0716 - accuracy: 0.3498 - val_loss: 2.1291 - val_accuracy: 0.1313
Epoch 745/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0811 - accuracy: 0.3572 - val_loss: 2.1572 - val_accuracy: 0.1281
Epoch 746/800
21/21 [==============================] - 1s 49ms/step - loss: 0.0715 - accuracy: 0.3532 - val_loss: 2.1438 - val_accuracy: 0.1313
Epoch 747/800
21/21 [==============================] - 1s 48ms/step - loss: 0.0789 - accuracy: 0.3733 - val_loss: 2.1683 - val_accuracy: 0.1281
Epoch 748/800
21/21 [==============================] - 1s 48ms/step - loss: 0.0713 - accuracy: 0.3512 - val_loss: 2.1581 - val_accuracy:

Epoch 799/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0716 - accuracy: 0.3554 - val_loss: 2.1642 - val_accuracy: 0.1360
Epoch 800/800
21/21 [==============================] - 1s 47ms/step - loss: 0.0672 - accuracy: 0.3575 - val_loss: 2.1602 - val_accuracy: 0.1352


In [6]:
from keras.models import load_model
training_model = load_model('model/training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
          #Predicting output tokens with probabilities and states
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
          if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
    #Update the target sequence
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [7]:
class ChatBot:
  negative_responses = ("no", "negativo", "nada", "nop", "nopi")
  exit_commands = ("chao", "adios", "nos vemos",
                 "suerte", "chaito", "bye", "chaolin")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hola, bienvenido en que te puedo ayudar?\n")
    
    if user_response in self.negative_responses:
      print("Ok, perfecto, que tengas un hermoso día!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Que tengas un hermoso día!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Hola
 nuestra entrada es yuquitas
Tienes pizza?
 claro que si necesitas que
de peperoni
 para tomar fines mazamorra
Y gaseosa?
 si el precio varia segun
chao
Que tengas un hermoso día!


In [ ]:
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Chao


In [9]:
chatbot.generate_response("que cuesta la arepa de huevo?")

' la arepa tiene un costo de 2000 pesos'